<a href="https://colab.research.google.com/github/tracydo99/bus118/blob/main/Prompt_Engineering_Customer_Service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet openai

import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "#####"
client = OpenAI()
MODEL = "gpt-4o-mini"

In [ ]:
from datetime import datetime
from typing import Dict, Any

def call_llm(system: str, user: str) -> str:
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role":"system","content":system},
                  {"role":"user","content":user}],
        temperature=0.2,
    )
    return resp.choices[0].message.content.strip()

Ticket = Dict[str, Any]

def step_1_classify_intent(ticket: Ticket) -> Ticket:
    system = """You are a senior airline customer support triage agent.
Classify the customer's request into ONE of:
booking, flight_status, change_itinerary, cancel_refund, baggage, seat_assignment,
special_assistance, loyalty_program, other.

Also extract any entities if present:
pnr (record locator), passenger_name, email, phone, flight_number, date,
origin_airport, destination_airport, baggage_tag, seat, loyalty_id.

Return a tight JSON object with keys: intent, entities (dict), confidence (0-1), notes.
If unsure, pick the closest category and note uncertainty."""
    user = f"""Customer message:
{ticket['message']}

Return JSON only."""
    ticket["triage"] = call_llm(system, user)
    return ticket

def step_2_collect_missing(ticket: Ticket) -> Ticket:
    system = """You are a polite airline support agent.
Ask ONLY for the essential missing details to move the case forward.
Keep it empathetic, ≤3 sentences. If requesting multiple items, use a single bullet list.
Prefer airline terms (PNR/record locator, flight number, travel date, origin/destination)."""
    user = f"""Known so far (from triage):
{ticket['triage']}

Based on this, ask the customer for any still-missing essentials (e.g., PNR, last name, flight number and date, origin/destination, baggage tag). Do not repeat info we already have."""
    ticket["followup_question"] = call_llm(system, user)
    return ticket

def step_3_resolution(ticket: Ticket) -> Ticket:
    system = """You are an airline resolution specialist.
Write a clear, courteous reply that:
- Acknowledges the issue with empathy,
- States the next concrete action(s) the airline can take or what will happen after receiving details,
- Includes any important constraints (fare rules, refund timelines, baggage tracing steps) as brief plain-English notes,
- Uses short paragraphs and, if helpful, one brief bullet list,
- ≤140 words, no legalese, no internal jargon."""
    user = f"""Context:
TRIAGE (JSON): {ticket['triage']}
FOLLOW-UP ASK (if needed): {ticket['followup_question']}

Draft a customer-facing reply suitable for airline customer service."""
    ticket["draft_reply"] = call_llm(system, user)
    return ticket

def run_chain(customer_message: str) -> Ticket:
    t = {"received_at": datetime.utcnow().isoformat(), "message": customer_message}
    for step in (step_1_classify_intent, step_2_collect_missing, step_3_resolution):
        t = step(t)
    return t

In [ ]:
msg = """Hi, I need to change my flight 145 on Oct 20 from SFO to JFK. My name is Tracy Do, and my PNR is 28TH7Z. Can you help me rebook to a later evening flight? Thank you."""
res = run_chain(msg)
print("=== TRIAGE ===\n", res["triage"], "\n")
print("=== FOLLOW-UP QUESTION ===\n", res["followup_question"], "\n")
print("=== DRAFT REPLY ===\n", res["draft_reply"])

/tmp/ipython-input-654895703.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  t = {"received_at": datetime.utcnow().isoformat(), "message": customer_message}


=== TRIAGE ===
 {
  "intent": "change_itinerary",
  "entities": {
    "pnr": "28TH7Z",
    "passenger_name": "Tracy Do",
    "flight_number": "145",
    "date": "2023-10-20",
    "origin_airport": "SFO",
    "destination_airport": "JFK"
  },
  "confidence": 1,
  "notes": ""
} 

=== FOLLOW-UP QUESTION ===
 Thank you for reaching out, Tracy. To assist you with changing your itinerary, could you please provide the following details?

- New travel date
- New destination airport (if applicable) 

I appreciate your cooperation! 

=== DRAFT REPLY ===
 Dear Tracy,

Thank you for reaching out regarding your itinerary change for flight 145 on October 20th. I understand how important it is to adjust your travel plans, and I'm here to help.

To proceed with changing your itinerary, could you please provide the following details?

- Your new travel date
- Any new destination airport (if applicable)

Once I have this information, I can assist you further. Please note that changes may be subject to f